In [1]:
pip install pandas matplotlib seaborn


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ------------------- -------------------- 3.9/8.1 MB 19.1 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 20.0 MB/s  0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 2.3/2.3 MB 22.4 MB/s  0:00:00

   ---------------------------------------- 0/7 [pyparsing]
   ----------- ---------------------------- 2/7 [fonttools]
   ----------- ---------------------------- 2/7 [fonttools]
   ----------- ---------------------------- 2/7 [fonttools]
   ----------- ---------------------------- 2/7 [fonttools]
   ----------- ---------------------------- 2/7 [fonttools]
   ----------- ---------------------------- 2/7 [fonttools]
   ----------- ---------------------------- 2/7 [fonttools]
   ----------- ---------------------------- 2/7 [fonttools]
   ----------- ---

In [2]:
from pathlib import Path
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
AUG_DIR = Path(r"C:\Users\ADMIN\Downloads\emotion classification\data_emotions")  # <-- change if needed
CSV_IN  = Path(r"C:\Users\ADMIN\Downloads\emotion classification\data_emotions\augmented\train_with_aug.csv")
OUT_DIR = Path(r"C:\Users\ADMIN\Downloads\emotion classification\data_emotions\augmented/EDA"); 
OUT_DIR.mkdir(parents=True, exist_ok=True)


In [7]:
ORDERED_CLASSES = ["happy","sad","anger","neutral","disgust","surprise","fear"]
SAVE_DPI = 180
# ------------------------------------------

sns.set_theme(style="whitegrid", context="talk")

# --- Load & normalize columns ---
df = pd.read_csv(CSV_IN)
df.columns = [c.strip().lower() for c in df.columns]
needed = {"resolved_path","image","label","label_norm","label_id"}
missing = needed - set(df.columns)
assert not missing, f"CSV is missing columns: {missing}"

# --- Detect augmented rows & parse tags (matches the augment filename pattern we used) ---
pat = re.compile(r"_r(?P<rot>-?\d+)_(?P<flip>hf|orig)_bc(?P<bc>[a-z]+)_c(?P<crop>\d+)", re.I)

def parse_aug(name: str):
    m = pat.search(name)
    if not m:
        return pd.Series({"is_augmented": False, "rot": None, "flip": None, "bc": None, "crop": None})
    return pd.Series({
        "is_augmented": True,
        "rot": int(m.group("rot")),
        "flip": "flipped" if m.group("flip").lower()=="hf" else "original",
        "bc": m.group("bc").lower(),        # dark / orig / bright
        "crop": int(m.group("crop"))
    })

tags = df["image"].astype(str).apply(parse_aug)
df = pd.concat([df, tags], axis=1)



## Class counts 

In [9]:
counts = (
    df["label_norm"].value_counts()
      .reindex(ORDERED_CLASSES, fill_value=0)
      .rename_axis("label_norm")
      .reset_index(name="count")
)
plt.figure(figsize=(10,6))
sns.barplot(data=counts, x="label_norm", y="count", order=ORDERED_CLASSES, edgecolor="black", errorbar=None)
plt.title("Class distribution (train_with_aug)")
plt.xlabel("Class"); plt.ylabel("# images")
plt.tight_layout()
plt.savefig(OUT_DIR / "class_counts_bar.png", dpi=SAVE_DPI); plt.close()


## Class percentage 

In [10]:
counts["percent"] = (counts["count"] / counts["count"].sum() * 100)
plt.figure(figsize=(10,6))
sns.barplot(data=counts, x="label_norm", y="percent", order=ORDERED_CLASSES, edgecolor="black", errorbar=None)
plt.title("Class distribution (%) (train_with_aug)")
plt.xlabel("Class"); plt.ylabel("Percent")
plt.tight_layout()
plt.savefig(OUT_DIR / "class_percent_bar.png", dpi=SAVE_DPI); plt.close()

## Original vs Augmented per class

In [11]:
stack = (
    df.groupby(["label_norm","is_augmented"])
      .size().reset_index(name="count")
      .assign(kind=lambda x: x["is_augmented"].map({False:"original", True:"augmented"}))
)
stack["label_norm"] = pd.Categorical(stack["label_norm"], categories=ORDERED_CLASSES, ordered=True)
plt.figure(figsize=(12,6))
sns.barplot(data=stack, x="label_norm", y="count", hue="kind", order=ORDERED_CLASSES,
            edgecolor="black", errorbar=None)
plt.title("Original vs Augmented per class")
plt.xlabel("Class"); plt.ylabel("# images")
plt.legend(title="")
plt.tight_layout()
plt.savefig(OUT_DIR / "orig_vs_aug_per_class.png", dpi=SAVE_DPI); plt.close()
